### Fetching API Key

In [1]:
from decouple import config
openai_api_key=config("OPENAI_API_KEY")

In [2]:
# print(openai_api_key)

### Pipeline for converting raw unstructed data into a QA chain
#### Indexing

1. Loading: Initially, the data needs to be loaded. Unstructured data can be sourced from various platforms. Utilize the LangChain Integration Hub to explore the complete range of loaders. Each loader outputs the data as a LangChain Document.

2. Splitting: Text splitters segment the Documents into specified sizes.

3. Storage: A storage solution, often a vector store, is used to house and sometimes embed the splits.

<img src="https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png" width="1000" height="450">

#### Retrieval and Generation

4. Retrieval: The application fetches the splits from the storage, usually based on embeddings similar to the input question.

5. Generation: A Language Model (LLM) generates an answer using a prompt that incorporates both the question and the retrieved data.

<img src="https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png" width="1000" height="450">

#### Bonous
6. Conversation (Extension): To facilitate multi-turn conversations, Memory can be added to the QA chain.

### Q & A pipeline RAG

Referrence: [Langchain](https://python.langchain.com/docs/use_cases/question_answering/)

* RAG is a technique for augmenting LLM knowledge with additional data

### Step 1: Loading the Document

In [3]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://aws.amazon.com/vi/what-is/langchain/")

corpus = loader.load()

In [4]:
# print(corpus)

### Step 2: Splitting the Document into Chunks

In [5]:
import tiktoken

# Set up token encoding for the GPT-3.5 Turbo model
tiktoken.encoding_for_model('gpt-3.5-turbo')

<Encoding 'cl100k_base'>

In [6]:
tokenizer = tiktoken.get_encoding('cl100k_base')

# Define a function to calculate the token length of a given text
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

# tiktoken_len("Dentin decay: Dentin is the layer just beneath your tooth enamel.")
tiktoken_len("LangChain là một khung mã nguồn mở để xây dựng các ứng dụng dựa trên các mô hình ngôn ngữ lớn (LLM).")

49

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Initialize the text splitter with specified parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = tiktoken_len
)
"""
Params:
    1. chunk_size: size of each chunk of text.
    2. chunk_overlap: each chunk will overlap with the previous chunk.
    3. length_function: the length of the text
"""

'\nParams:\n    1. chunk_size: size of each chunk of text.\n    2. chunk_overlap: each chunk will overlap with the previous chunk.\n    3. length_function: the length of the text\n'

In [8]:
# Split the loaded document into smaller chunks
chunks = text_splitter.split_documents(corpus)

In [9]:
chunks

[Document(page_content='LangChain là gì? – Giải thích về LangChain – AWS\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Chuyển đến nội dung chính\n\n\n\n\n\nNhấp vào đây để quay lại trang chủ Amazon Web Services', metadata={'source': 'https://aws.amazon.com/vi/what-is/langchain/', 'title': 'LangChain là gì? – Giải thích về LangChain – AWS', 'description': 'Khái niệm về LangChain, cách thức và lý do các doanh nghiệp sử dụng LangChain, cũng như cách sử dụng LangChain với AWS.', 'language': 'vi-VN'}),
 Document(page_content='Nhấp vào đây để quay lại trang chủ Amazon Web Services\n\n\n\nLiên hệ với chúng tôi\n Hỗ trợ\xa0 \nTiếng Việt\xa0\nTài khoản của tôi\xa0\n\n\n\n\n Đăng nhập\n\n\n  Tạo tài khoản AWS', metadata={'source': 'https://aws.amazon.com/vi/what-is/langchain/', 'title': 'LangChain là gì? – Giải thích về LangChain – AWS', 'description': 'Khái niệm về LangChain, cách thức và lý do các doanh nghiệp sử dụng

In [10]:
len(chunks)

95

### Step 3: Storing the Vector Embeddings in Vector Database

In [11]:
# Update Jupyter and ipywidgets

# %pip install jupyter ipywidgets
# !jupyter nbextension enable --py widgetsnbextension

In [12]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "vinai/phobert-base"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

No sentence-transformers model found with name /home/sanslab/.cache/torch/sentence_transformers/vinai_phobert-base. Creating a new one with MEAN pooling.


In [13]:
embed = hf.embed_documents(texts=['abc', 'def'])

# Print the length of one of the embeddings to check its dimensions
print(len(embed[1]))

768


In [14]:
from langchain.vectorstores import Chroma

vectordb = Chroma.from_documents(chunks, hf)

In [15]:
vectordb.similarity_search('Langchain là công cụ xây dựng API cho các mô hình LLMs', k = 3)

[Document(page_content='Dưới đây là một ví dụ về chức năng chuỗi chatbot trả về chi tiết sản phẩm bằng nhiều ngôn ngữ.\nchain([\nretrieve_data_from_product_database().\nsend_data_to_language_model().\n\xa0\xa0\xa0format_output_in_a_list().\n\xa0\xa0translate_output_in_target_language()\n])', metadata={'source': 'https://aws.amazon.com/vi/what-is/langchain/', 'title': 'LangChain là gì? – Giải thích về LangChain – AWS', 'description': 'Khái niệm về LangChain, cách thức và lý do các doanh nghiệp sử dụng LangChain, cũng như cách sử dụng LangChain với AWS.', 'language': 'vi-VN'}),
 Document(page_content='Sau đó, các nhà phát triển sử dụng các khối xây dựng chuỗi hoặc Ngôn ngữ biểu thức LangChain (LCEL) để soạn chuỗi bằng các lệnh lập trình đơn giản. Hàm chain() truyền các đối số của liên kết đến các thư viện. Lệnh execute()', metadata={'source': 'https://aws.amazon.com/vi/what-is/langchain/', 'title': 'LangChain là gì? – Giải thích về LangChain – AWS', 'description': 'Khái niệm về LangChain

### Step 4: Retrieval and Generate

In [16]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# Initialize a language model with ChatOpenAI
llm = ChatOpenAI(model_name = 'gpt-3.5-turbo', temperature=0.6)

# Initialize a RetrievalQA chain with the language mode and vector database retrieval
qa_chain = RetrievalQA.from_chain_type(llm, retriever= vectordb.as_retriever())

# Pass a query to the QA chain to generate an answer
qa_chain({'query': 'Langchain là gì?'})

{'query': 'Langchain là gì?',
 'result': 'LangChain là một thư viện Python cho phép các nhà phát triển truy xuất dữ liệu từ cơ sở dữ liệu sản phẩm, gửi dữ liệu đó đến mô hình ngôn ngữ, định dạng kết quả và dịch kết quả sang ngôn ngữ mục tiêu khác nhau. Để sử dụng LangChain, bạn cần cài đặt thư viện bằng lệnh "pip install langchain" trong môi trường Python của mình.'}

In [17]:
qa_chain({'query': 'Tại sao chúng ta nên sử dụng Langchain cho các mô hình LLMs lớn?'})

{'query': 'Tại sao chúng ta nên sử dụng Langchain cho các mô hình LLMs lớn?',
 'result': 'LangChain là một giao diện gắn kết các thành phần AI như Amazon Bedrock, Amazon Kendra, Amazon SageMaker JumpStart và LLM (Large Language Models) lại với nhau. Khi sử dụng LangChain, bạn có thể xây dựng các ứng dụng trí tuệ nhân tạo tạo sinh (AI tạo sinh) có độ chính xác cao trên dữ liệu doanh nghiệp. Việc sử dụng LangChain giúp tạo ra các mô hình LLM lớn với sự linh hoạt và hiệu quả, đồng thời giúp các nhà phát triển soạn chuỗi bằng các lệnh lập trình đơn giản thông qua Ngôn ngữ biểu thức LangChain (LCEL). Điều này giúp tăng cường khả năng phát triển và triển khai các mô hình LLM lớn một cách hiệu quả và dễ dàng.'}

In [18]:
qa_chain({'query': 'Đại học Bách Khoa Hà Nội ở đâu?'})

{'query': 'Đại học Bách Khoa Hà Nội ở đâu?',
 'result': 'Xin lỗi, tôi không biết câu trả lời cho câu hỏi của bạn.'}

In [ ]:
# !python -m pip install ragas

In [ ]:
from ragas import evaluate
from datasets import Dataset
import os

os.environ["OPENAI_API_KEY"] = "your-openai-key"

# prepare your huggingface dataset in the format
# Dataset({
#     features: ['question', 'contexts', 'answer', 'ground_truth'],
#     num_rows: 25
# })

dataset: Dataset

results = evaluate(dataset)
# {'context_precision': 0.817,
# 'faithfulness': 0.892, 
# 'answer_relevancy': 0.874}